In [9]:
from gerrychain import Graph, Election, updaters, GeographicPartition
import statistics

In [10]:
def print_to_file(writing_str):
    with open('outlier_analysis_output.txt', 'a') as f:
        print(writing_str, file=f)

In [11]:
g = Graph.from_file("./NY-lab/NY.shp")

NUM_DISTRICTS = 26

/Users/travisoleson/anaconda3/lib/python3.11/site-packages/gerrychain/graph/graph.py:388: UserWarning: Found islands (degree-0 nodes). Indices of islands: {134451, 134452}
  warnings.warn(
/Users/travisoleson/anaconda3/lib/python3.11/site-packages/gerrychain/graph/graph.py:262: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  areas = df.geometry.area.to_dict()


In [12]:
elections = [
    Election("PRE20", {"Democratic": "G20PRED", "Republican": "G20PRER"}),
    # Election("SEN18", {"Democratic": "G18SEND", "Republican": "G18SENR"}),
    # Election("GOV18", {"Democratic": "G18GOVD", "Republican": "G18GOVR"}),
    # Election("COM18", {"Democratic": "G18COMD", "Republican": "G18COMR"}),
    # Election("ATG18", {"Democratic": "G18ATGD", "Republican": "G18ATGR"}),
]
my_updaters = {"population": updaters.Tally("TOTPOP", alias="population")}
election_updaters = {election.name: election for election in elections}
my_updaters.update(election_updaters)


In [13]:
initial_partition = GeographicPartition(g,
                                        assignment= "CD", updaters=my_updaters)

In [14]:
## MM is directly proportional to the gerymandering of the districting plan

def mm(part, election, party):
    if party == "Democratic":
    # Get the vote totals for the Democratic and Republican parties
        votes = part[election].percents("Democratic")
    else:
        votes = part[election].percents("Republican")

    # # Calculate the mean-median difference
    mean_median_diff = statistics.median(votes) - statistics.mean(votes)

    return mean_median_diff

In [15]:
for election in elections:
    print_to_file("---------")
    print_to_file("election -> {}".format(election.name))
    print_to_file("---------")
    for party in ["Democratic", "Republican"]:
        print_to_file("party -> {}".format(party))
        diff = mm(initial_partition, election.name, party)
        print_to_file("MM Difference -> {}".format(diff))
        print_to_file("---------")

    print_to_file("\n")

In [16]:
# Efficiency Gap

def eg(part, election):
    # Get the vote totals for the Democratic and Republican parties
    dem_votes = list(part[election].totals_for_party['Democratic'].values())
    rep_votes = list(part[election].totals_for_party['Republican'].values())

    winning_areas = set()

    # Get winning areas
    for i in range(len(dem_votes)):
        if dem_votes[i] >= rep_votes[i]:
            winning_areas.add(i)

    # Calculate the number of wasted votes for the Democratic and Republican parties
    dem_wasted_votes = 0
    rep_wasted_votes = 0

    for i in range(len(dem_votes)):
        if i in winning_areas:
            dem_wasted_votes += dem_votes[i] - (0.5 * (dem_votes[i] + rep_votes[i]))
            rep_wasted_votes += rep_votes[i]
        else:
            dem_wasted_votes += dem_votes[i]
            rep_wasted_votes += rep_votes[i] - (0.5 * (dem_votes[i] + rep_votes[i]))

    # Calculate the efficiency gap
    efficiency_gap = (rep_wasted_votes - dem_wasted_votes) / sum(dem_votes + rep_votes)

    return efficiency_gap


In [17]:
for election in elections:
    print_to_file("---------")
    print_to_file("election -> {}".format(election.name))
    print_to_file("---------")
    diff = eg(initial_partition, election.name)
    print_to_file("Efficiency Gap -> {}".format(diff))
    print_to_file("---------")

    print_to_file("\n")

In [18]:
## Cut Edges
from gerrychain.tree import recursive_tree_part

In [19]:
total_population = sum([g.nodes[node]["TOTPOP"] for node in g.nodes()])

pop_str = "Total Population:" + str(total_population)
print_to_file(pop_str)

In [20]:
districting_plan = recursive_tree_part(g, range(NUM_DISTRICTS), total_population/NUM_DISTRICTS, "TOTPOP", 0.02, 10)

cut_edges = len([edge for edge in g.edges() if districting_plan[edge[0]] != districting_plan[edge[1]]])

pop_str = "Cut Edges:" + str(cut_edges)
print_to_file(pop_str)

In [21]:
from collections import defaultdict

democrat_winning_info = defaultdict(list)
republican_winning_info = defaultdict(list)
## Election: [districts won]

for election in elections:
    # election.name
    dem_votes = initial_partition[election.name].percents("Democratic")
    for district_id, district_votes in enumerate(dem_votes):
        if district_votes > 0.5:
            democrat_winning_info[election.name].append(district_id)
        else:
            republican_winning_info[election.name].append(district_id)

In [22]:
print_to_file(democrat_winning_info)

In [ ]:
def part_bias(part, election):
    """
    TODO: Calculate the partisan bias of the districting map corresponding to partition `part`.
    Return the value of the partisan bias, from the perspective of the Democratic party.
    """

    # Get the vote totals for the Democraticparties
    dem_votes = part[election].percents("Democratic")

    dem_vote_share = 0

    for i in range(len(dem_votes)):
        if dem_votes[i] >= statistics.mean(dem_votes): ## Higher than the mean
            dem_vote_share += 1

    s_v = dem_vote_share / len(dem_votes)

    # Calculate the partisan bias
    partisan_bias = s_v - 0.5

    return partisan_bias
